# Notebook : Modélisation avec Régression Linéaire

**Objectif :** Entraîner un modèle de régression linéaire pour prédire le `nutriscore_grade`, évaluer ses performances, et extraire des paramètres statistiques pour analyse.

---

## Étape 1 : Importer les bibliothèques nécessaires et charger les données


In [18]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
import os
import joblib

### Charger le fichier de données finales avec les variables sélectionnées


In [19]:

file_path = 'note-books/phase_2_training/04_correlation_analysis/data_final_features.csv'
df = pd.read_csv(file_path)

# Afficher un aperçu des données
print("Aperçu des premières lignes du jeu de données sélectionné :")
df.head()

Aperçu des premières lignes du jeu de données sélectionné :


,fat_100g,saturated-fat_100g,energy-kcal_100g,sugars_100g,salt_100g,proteins_100g,product_name,brands,categories,nutriscore_grade
0,5.1,0.8,277.0,5.0,1.17,7.4,Pain de mie extra moelleux sans sucres ajoutÃ©s,"Lidl,MaÃ®tre Jean Pierre",Pains de mie,b
1,14.7,8.1,361.0,3.2,0.68,0.0,Vegan Protein Bar Almond Cookie Dough,IronMaxx,NahrungsergÃ¤nzungsmittel,d
2,5.3,3.4,93.0,6.4,0.20,4.9,cuajada,Milbona,Lcteos,b
3,20.0,0.0,247.0,0.8,2.50,16.0,Salad cheese Greek Style,Milbona,Lcteos,d
4,1.0,0.5,377.0,3.7,0.08,13.6,Fusilli,Lidl,Pflanzliche Lebensmittel und GetrÃ¤nke,a


**Explication :**
- Le fichier `data_final_features.csv` contient les variables nutritionnelles pour prédire `nutriscore_grade` et des identifiants de produit.

---

## Étape 2 : Préparer les données pour la modélisation

Nous séparons la variable cible (`nutriscore_grade`) des variables explicatives, puis divisons les données en ensembles d’entraînement et de test (80/20).

In [20]:
# Encoder `nutriscore_grade` en valeurs numériques pour la prédiction
df['nutriscore_grade_encoded'] = df['nutriscore_grade'].map({'a': 0, 'b': 1, 'c': 2, 'd': 3, 'e': 4})

# Définir les variables explicatives et la cible
# Vérifiez d'abord quelles colonnes d'énergie sont disponibles
# Utiliser seulement 'energy-kcal_100g' si 'energy_100g' est absent
energy_column = 'energy_100g' if 'energy_100g' in df.columns else 'energy-kcal_100g'

# Redéfinir les variables explicatives en fonction de la disponibilité des colonnes
X = df[['fat_100g', 'saturated-fat_100g', energy_column, 'sugars_100g', 'salt_100g', 'proteins_100g']]
y = df['nutriscore_grade_encoded']  # Utiliser la version encodée comme cible

# Diviser les données en ensembles d'entraînement et de test (80/20)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print("Ensemble d'entraînement :", X_train.shape)
print("Ensemble de test :", X_test.shape)

Ensemble d'entraînement : (133647, 6)
Ensemble de test : (33412, 6)


**Explication :**
- Nous utilisons les variables nutritionnelles dans `X` pour prédire `nutriscore_grade`.
- `train_test_split` divise les données en 80 % pour l’entraînement et 20 % pour le test.

---

## Étape 3 : Entraîner le modèle de régression linéaire

In [21]:
# Initialiser le modèle de régression linéaire
model = LinearRegression()

# Entraîner le modèle sur l'ensemble d'entraînement
model.fit(X_train, y_train)

print("Modèle de régression linéaire entraîné.")

Modèle de régression linéaire entraîné.


**Explication :**
- `LinearRegression()` initialise le modèle.
- `model.fit(X_train, y_train)` ajuste le modèle aux données d’entraînement.

---

## Étape 4 : Évaluer les performances du modèle

Nous évaluons les performances en utilisant le **Mean Squared Error (MSE)** et le **coefficient de détermination (R²)** sur l’ensemble de test.

In [22]:
# Prédire les valeurs sur l'ensemble de test
y_pred = model.predict(X_test)

# Calculer le Mean Squared Error (MSE)
mse = mean_squared_error(y_test, y_pred)

# Calculer le coefficient de détermination R²
r2 = r2_score(y_test, y_pred)

print("Évaluation du modèle :")
print(f"Mean Squared Error (MSE) : {mse}")
print(f"Coefficient de détermination (R²) : {r2}")

Évaluation du modèle :
Mean Squared Error (MSE) : 1.0783356102786747
Coefficient de détermination (R²) : 0.4032539579473836


**Explication :**
- `model.predict(X_test)` génère des prédictions.
- `mean_squared_error` et `r2_score` évaluent la précision du modèle.

---

## Étape 5 : Extraire les paramètres statistiques du modèle

Les paramètres statistiques incluent les coefficients des variables, l’intercept, et les résidus (erreurs de prédiction).

In [23]:
# Coefficients des variables et intercept
coefficients = model.coef_
intercept = model.intercept_

# Calcul des résidus (différence entre les valeurs prédites et réelles)
residuals = y_test - y_pred

print("Paramètres statistiques du modèle :")
print(f"Intercept : {intercept}")
print("Coefficients :")
for feature, coef in zip(X.columns, coefficients):
    print(f"{feature} : {coef}")

print("\nAperçu des résidus :")
print(residuals[:5])  # Affiche les cinq premiers résidus pour exemple

Paramètres statistiques du modèle :
Intercept : 1.2730532391101386
Coefficients :
fat_100g : 0.02685592754921234
saturated-fat_100g : 0.06416192347869022
energy-kcal_100g : 0.001302954398841145
sugars_100g : 0.017150569664145767
salt_100g : 0.34865011810405594
proteins_100g : -0.04239648898691274

Aperçu des résidus :
113533   -0.331524
21047     2.128490
6075      0.044995
96947     1.628544
6273      1.677489
Name: nutriscore_grade_encoded, dtype: float64


**Explication :**
- `model.intercept_` et `model.coef_` donnent respectivement l'intercept et les coefficients de chaque variable explicative.
- Les résidus montrent l'écart entre les valeurs observées et prédites.

---

## Étape 6 : Sauvegarder le modèle, les métriques de performance et les paramètres statistiques

In [24]:
# Définir le dossier de sortie
output_folder = 'note-books/phase_2_training/05_model_baseline'
os.makedirs(output_folder, exist_ok=True)

# Sauvegarder le modèle de régression linéaire
model_path = os.path.join(output_folder, 'linear_regression_model.joblib')
joblib.dump(model, model_path)

# Sauvegarder les métriques de performance
metrics_path = os.path.join(output_folder, 'performance_metrics.txt')
with open(metrics_path, 'w') as f:
    f.write(f"Mean Squared Error (MSE) : {mse}\n")
    f.write(f"Coefficient de détermination (R²) : {r2}\n")

# Sauvegarder les paramètres statistiques du modèle
params_path = os.path.join(output_folder, 'model_parameters.txt')
with open(params_path, 'w') as f:
    f.write(f"Intercept : {intercept}\n")
    f.write("Coefficients :\n")
    for feature, coef in zip(X.columns, coefficients):
        f.write(f"{feature} : {coef}\n")
    f.write("\nAperçu des résidus :\n")
    f.write("\n".join(map(str, residuals[:5])))  # Sauvegarde les cinq premiers résidus pour exemple

print(f"Modèle et métriques sauvegardés dans le dossier : {output_folder}")

Modèle et métriques sauvegardés dans le dossier : note-books/phase_2_training/05_model_baseline



**Explication :**
- `joblib.dump(model, model_path)` enregistre le modèle.
- Les métriques et paramètres statistiques sont sauvegardés dans des fichiers texte pour référence.

---

### Résumé

Dans ce notebook, nous avons :
1. Chargé et préparé les données pour la modélisation.
2. Divisé les données en ensembles d’entraînement et de test (80/20).
3. Entraîné un modèle de régression linéaire pour prédire `nutriscore_grade`.
4. Évalué les performances avec le MSE et R² pour établir une première référence.
5. Extrait et sauvegardé les paramètres statistiques du modèle, incluant les coefficients, l'intercept, et un aperçu des résidus.
